# Web Scraper de Indicadores de Ativos

Requirements:
- gspread 4.0.1
- pandas 1.3.5
- requests 2.23.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture
!pip install --upgrade beautifulsoup4

In [ ]:
# Usar essa instalação ou adaptar o código
%%capture
!pip install https://github.com/burnash/gspread/releases/download/v4.0.1/gspread-4.0.1.tar.gz

In [ ]:
import gspread
from gspread_dataframe import set_with_dataframe

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
requests.__version__

In [ ]:
# Definir a lista de Ativos que terão seus indicadores coletados

list_acao = [
  'BBSE3',
  'CCRO3',
  'BBAS3',
  'SAPR4',
  'MGLU3',
  'BBDC4',
  'ITSA4',
  'DIRR3',
  'CSMG3',
  'GRND3',
  'KLBN4',
  'TRIS3',
  'YDUQ3',
  'ELET3',
  'ENBR3',
  'TAEE4',
  'CMIG4',
  'AESB3',
  'SHUL4',
  'FESA4',
  'STBP3',
  'AZUL4',
  'OIBR4',
  'GOLL4',
  'OIBR3',
  'COGN3',
  'ROMI3',
  'ENAT3',
  'VIVT3',
  'PCAR3',
  'CSMG3',
  'MRVE3',
  'TRPL4',
  'CYRE3',
  'EGIE3',
  'IRBR3',
  'NGRD3',
  'LAME4',
  'VIIA3'
]

list_fii = [
  'RECT11',
  'BRCR11',
  'HSML11',
  'RECR11',
  'GGRC11',
  'HGRE11',
  'SDIL11',
  'RBFF11',
  'HCTR11',
  'MALL11',
  'TGAR11',
  'HFOF11',
  'HTMX11',
  'CPTS11',
  'KNRI11',
  'VISC11',
  'BCFF11',
  'VINO11',
  'VILG11',
  'MXRF11'
]

In [ ]:
# Definir as planílhas onde os dados serão salvos

spreadsheetId_acao = '11qJs0uNM0jnRB6t-FeJpiS6ZahXvzlDzxW6aZ6zhhOQ' #### PRECISA SER ALTERADO ####

spreadsheetId_fii = '1BG4BUFA_eo1TZ21Z2g4k2jAvuwVT1CV3lv4kiohalQ0'   #### PRECISA SER ALTERADO ####

In [ ]:
class AssetScraper():
    def __init__(self, assetList, spreadsheetId):
        self.assetList = assetList
        self.spreadsheetId = spreadsheetId

    def exportAssetDataframe(self): 
            if self.assetDataframe.empty:
                raise RuntimeError("Build the asset sheet first to export it")

            #### ABAIXO COLOCAR O CAMINHO DO LOCAL ONDE SUA CHAVE DE PROJETO ESTÁ LOCALIZADA ####
            gc = gspread.service_account(filename='/content/drive/MyDrive/Investimentos/Controles/projeto-web-scrapper-8d9f697f95a8.json') #### PRECISA SER ALTERADO ####

            # Acessa o google sheet
            sheet = gc.open_by_key(self.spreadsheetId)
            worksheet = sheet.sheet1

            # Limpa o conteúdo prévio
            worksheet.clear()

            # Adiciona os dados a spreadsheet
            set_with_dataframe(worksheet, self.assetDataframe) #-> THIS EXPORTS YOUR DATAFRAME TO THE GOOGLE SHEET

In [ ]:
class FiiScraper(AssetScraper):
    
    def __init__(self, assetList, spreadsheetId):
        super().__init__(assetList, spreadsheetId)
      
        self.baseUrl = 'https://www.fundsexplorer.com.br/ranking'

        page = requests.get(self.baseUrl)
        worksheet = pd.read_html(page.text)[0]

        # Mantém apenas os FIIs que estão na lista
        worksheet = worksheet[worksheet['Códigodo fundo'].isin(self.assetList)]

        # Tratamento da coluna p/vp pra porque vem multiplicada por 100
        worksheet['P/VPA'] = worksheet['P/VPA']/100

        # Manter apenas as necessárias
        worksheet = worksheet[[
        'Códigodo fundo',
        'Setor',
        'Preço Atual',
        'Liquidez Diária',
        'DividendYield',
        'DY (12M)Média',
        'DY Ano',
        'DY (12M)Acumulado',
        'PatrimônioLíq.',
        'P/VPA',
        'VacânciaFísica',
        'VacânciaFinanceira',
        'QuantidadeAtivos'    
        ]]

        self.assetDataframe = worksheet

In [ ]:
class AcaoScraper(AssetScraper):
    
    def __init__(self, assetList, spreadsheetId):
        super().__init__(assetList, spreadsheetId) 
          
        self.baseUrl = 'https://statusinvest.com.br/acoes/'

        self.assetIndicators = {
            'P/VP': {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(1) > div > div:nth-child(4) > div > div > strong'},
            
            'DY12M': {'selector': '#main-2 > div:nth-child(4) > div > div.pb-3.pb-md-5 > div > div:nth-child(4) > div > div:nth-child(1) > strong'},

            "PL": {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(1) > div > div:nth-child(2) > div > div > strong'},

            "ROE": {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(4) > div > div:nth-child(1) > div > div > strong'},

            "ROIC": {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(4) > div > div:nth-child(3) > div > div > strong'},

            "M.LIQUIDA": {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(3) > div > div:nth-child(4) > div > div > strong'},

            "EV/EBITDA": {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(1) > div > div:nth-child(5) > div > div > strong'},

            "DIV.LIQUIDA/EBITDA": {'selector': '#indicators-section > div.indicator-today-container > div > div:nth-child(2) > div > div:nth-child(2) > div > div > strong'}
        }
        
        for indicator in self.assetIndicators:
            self.assetIndicators[indicator]['values'] = []

        for asset in self.assetList:
            print("Processing asset: %s ..." % asset)

            page = requests.get(self.baseUrl + asset)    
            soup = BeautifulSoup(page.content, 'html.parser')

            for indicator in self.assetIndicators:
                try: 
                    indicatorValue = soup.select(self.assetIndicators[indicator]['selector'])[0].string
                except IndexError:
                    indicatorValue = 'ERROR'
                self.assetIndicators[indicator]['values'].append(indicatorValue)

            print("........................................... Processing finished\n\n")

        # Monta o Dataframe
        DataframeColumns = {"Ativo": self.assetList}

        for indicator in self.assetIndicators:
            DataframeColumns[indicator] = self.assetIndicators[indicator]['values']
        self.assetDataframe = pd.DataFrame(DataframeColumns)

In [ ]:
acao_scraper = AcaoScraper(list_acao, spreadsheetId_acao)

In [ ]:
acao_scraper.assetDataframe

In [ ]:
acao_scraper.exportAssetDataframe()

In [ ]:
fii_scraper = FiiScraper(list_fii, spreadsheetId_fii)

In [ ]:
fii_scraper.assetDataframe

In [ ]:
fii_scraper.exportAssetDataframe()